In [1]:
import numpy as np
import pandas as pd

In [2]:
def rigid_transform_3D(A, B):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = -R @ centroid_A + centroid_B

    return R, t

In [7]:
gcp_scanner_df = pd.read_csv("control_pts_scanner.csv", sep=" ")
gcp_gps_df = pd.read_csv("control_pts_gps.csv", sep=" ")

In [8]:
gcp_gps_df

,x,y,z
0,502615.837416,616392.151322,89.587705
1,502616.256385,616469.262778,89.794408
2,502594.379667,616558.695956,89.171018
3,502524.669785,616587.489906,89.505680
4,502427.991902,616491.050144,91.123213
5,502381.602143,616512.968785,91.362726
6,502328.829466,616390.248678,91.884405
7,502372.644143,616373.135921,91.766289
8,502425.905650,616352.458757,91.566148
9,502481.402060,616330.819260,91.841752


In [9]:
gcp_scanner_df

,x,y,z
0,47.555287,-63.115917,111.269227
1,82.307232,5.700785,111.502192
2,102.612568,95.612989,110.734131
3,52.715164,152.089213,111.056402
4,-76.758738,108.568019,111.058551
5,-108.618449,148.824252,111.301430
6,-210.196179,61.455641,111.947384
7,-178.410512,26.841211,111.814020
8,-139.726660,-15.255742,111.581420
9,-99.436486,-59.155298,111.694845


In [10]:
A = np.array(gcp_scanner_df.loc[[0,4,8], :]).T
B = np.array(gcp_gps_df.loc[[0,4,8], :]).T

In [11]:
[ret_R, ret_t] = rigid_transform_3D(A, B)

# Compare the recovered R and t with the original
B2 = (ret_R@A) + ret_t

n = A.shape[1]

# Find the root mean squared error
err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)


In [12]:
rmse

0.3473114820088297

In [13]:
ret_R

array([[ 8.97127978e-01, -4.41680723e-01,  8.91799159e-03],
       [ 4.41703815e-01,  8.97160656e-01, -7.04494068e-04],
       [-7.68970974e-03,  4.57113225e-03,  9.99959986e-01]])

In [14]:
from math import asin, atan2, cos

theta1 = -asin(ret_R[2,0])
theta2 = np.pi - theta1

psi1 = atan2(ret_R[2,1] / cos(theta1), ret_R[2,2] / cos(theta1))
psi2 = atan2(ret_R[2,1] / cos(theta2), ret_R[2,2] / cos(theta2))

phi1 = atan2(ret_R[1,0] / cos(theta1), ret_R[0,0] / cos(theta1))
phi2 = atan2(ret_R[1,0] / cos(theta2), ret_R[0,0] / cos(theta2))

ret_Rotation_angle1 = 180 / np.pi * np.array([psi1, theta1, phi1])
ret_Rotation_angle2 = 180 / np.pi * np.array([psi2, theta2, phi2])

In [15]:
ret_t

array([[ 5.02543882e+05],
       [ 6.16427807e+05],
       [-2.10186087e+01]])

In [16]:
ret_Rotation_angle1

array([ 0.26191524,  0.44059226, 26.21347592])

In [17]:
before = np.array([502615.83741648926, 616392.1513218867, 89.587705])
after = np.array([502615.414561, 616392.108846, 89.591968])

before - after

array([ 0.42285549,  0.04247589, -0.004263  ])